In [1]:
print('MUST BE PYTHON 3 !!!!!!!!!!!!!!!!!!!!')

MUST BE PYTHON 3 !!!!!!!!!!!!!!!!!!!!


In [2]:


# Import modules
#import mpld3
%matplotlib inline
#mpld3.enable_notebook()
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime
import xarray as xr
from astropy.io import ascii
import pytz
# OS interaction
import sys
import os
import glob

In [3]:
# http://stackoverflow.com/questions/38987/how-can-i-merge-two-python-dictionaries-in-a-single-expression
def merge_two_dicts(x, y):
    '''Given two dicts, merge them into a new dict as a shallow copy.'''
    z = x.copy()
    z.update(y)
    return z

In [4]:
def RH_2_SPHM(RELHM, PRESS, TAIR):
    # Taken from Martyn Clark, SUMMMA code.
    #     IMPLICIT NONE

    #     REAL(DP), INTENT(IN)         :: RELHM     ! Relative Humidity     (%)
    #     REAL(DP), INTENT(IN)         :: PRESS     ! Pressure              (Pa)
    #     REAL(DP), INTENT(IN)         :: TAIR      ! Air temp

    #     REAL(DP)                     :: RELHM2SPHM ! Specific Humidity (g/g)

    #     REAL(DP)                     :: PVP       ! Partial vapour pressure at T  (Pa)
    #     REAL(DP)                     :: TDCEL     ! Dewpt in celcius      (C)
    #     !REAL(DP)                     :: DUM       ! Intermediate

    #     ! Units note :              Pa = N m-2 = kg m-1 s-2
    SATVPFRZ=     610.8       # Saturation water vapour pressure at 273.16K (Pa)
    TFREEZE = 273.15
    W_RATIO =       0.622     # molecular weight ratio of water to dry air (-)

    TDCEL = TAIR-TFREEZE
    PVP = RELHM * SATVPFRZ * np.exp( (17.27*TDCEL)/(237.30 + TDCEL) ) # Partial Vapour Press (Pa)
    RELHM2SPHM = (PVP * W_RATIO)/(PRESS - (1. - W_RATIO)*PVP)
    return RELHM2SPHM

In [5]:
# Directories
# time_dir     = 'Fortress_Pre_telemetry_QC_data'
# time_dir     = 'Fortress_2014_2016_QC_data'
# time_dir     = 'Fortress_2016_PostNov2014Storm_QC_data'
time_dir = 'Marmot Creek QC data_2005_16'


main_dir     = os.path.normpath(r'C:\Users\new356\Google Drive\Nov2014 Data QC Completed Data')
# MetaDataFile
meta_data_file  = os.path.join(main_dir,'CRHO_Station_lat_long_elevation.txt')
# Ascii input folder
dir_in     = os.path.join(main_dir, time_dir, 'QC_Data_ASCII_slim')
# netcdf output folder
dir_out    = os.path.join(main_dir, time_dir, 'QC_Data_netcdf')
# all data out
cfileout   = os.path.join(dir_out,time_dir) + '.nc'

In [6]:
# Define input format of ascii files
input_format = 'CRHO_TELM'

In [7]:
if input_format == 'CRHO_TELM':
    # Ascii data format info
    c_header = 4 # Header lines
    c_column_line = 1 # line where column names start
    c_delimiter = ','
    # time zone variables
    #tz_in = pytz.timezone('Etc/GMT-6')

In [8]:
# Get file in info
os.chdir(dir_in) # Move to input
content = glob.glob('*.txt') # Get list of files
num_files = len(content)

In [9]:
# Read in metadata (all stations)
metadata = pd.read_csv(meta_data_file,index_col='station')

In [10]:
metadata

,lat,long,elevation(m)
station,,,
BNS,50.820920,-115.213910,2099
PWL,50.825980,-115.198200,2100
FRS,50.838230,-115.215780,2330
CRN,50.821600,-115.206500,2205
BRP,50.760610,-115.367150,2260
FLG,50.830040,-115.228510,2565
PYT,51.685600,-116.544900,2237
BWH,51.635180,-116.490280,2421
CRG,50.821500,-115.206300,2211


In [11]:
# Initalize stuff
c_dict = {}
stations_all=[]
variables=[]
units_all = {}
time_index = {}

In [12]:
# Read in each file
for cfile in content:
    
    # Get current station name
    csta_name = cfile[0:3] # Take the first three letter abbreviation as the name
    print('Processing ' + csta_name)
    stations_all.append(csta_name)
    
    # Import data to pandas dataframe
    dat = ascii.read(cfile,header_start=c_column_line,data_start=c_header,delimiter=c_delimiter,exclude_names='N/A')
    datain = pd.DataFrame(dat.as_array())
    
    # Alternate method that drops duplicate (second variables) (Not correct but works for now)
    #print("Using temp fix to process dataframes with duplicate columns. DATA NOT CORRECT!!!!")
    #datain = pd.read_csv(content[4],header=c_column_line) #,mangle_dupe_cols=False)
    #datain.drop(datain.index[:2], inplace=True)
    #datain = datain[datain.columns.drop(datain.filter(regex='.1'))]
    #datain.columns
    
    
    # Replace -9999 with nan (recomended by netcdf)
    datain.replace(-9999,np.NaN,inplace=True)
    
    # Make TIMESTAMP the index
    datain['TIMESTAMP'] = datain['TIMESTAMP'].astype('datetime64[ns]')
    datain = datain.set_index('TIMESTAMP')
    

    ## Rename funky station names to common MVF names
    #datain.rename(columns=var_names, inplace=True)
    
    # Import header info 
    headerinfo = pd.read_csv(cfile,nrows=2,skiprows=1)
    units = headerinfo.loc[0,:].tolist() # Grab first row of dataframe (units)
    units = units[1:] # Remove first value which is the units of the timestamp
    units_dic = dict(zip(datain.columns,units)) # Dictionary of variable:units for this stations
    units_all = merge_two_dicts(units_all, units_dic) # Merge dictoinaries together (units_dic overwrites any units_all)
    
    # Loop through all variables for this station
    c_variables = datain.columns
    variables.extend(c_variables.values) # Store all variables for use latter
    for c_var in c_variables:
        c_dict[(csta_name,c_var)] = pd.DataFrame(datain[c_var])
        c_dict[(csta_name,c_var)].columns = [c_var]
        c_dict[(csta_name,c_var)].index  = datain.index
        
    # Save time index for each station (need to fill in missing variables later)
    time_index[csta_name] = datain.index

Processing CNT
Processing FSR
Processing HMW
Processing LLF
Processing UCT
Processing UPC
Processing UPF
Processing VVW


In [13]:
# Drop any vars with 'drop' in them (unwanted variables)
vars_new = [Cv2 for Cv2 in variables if 'drop' not in Cv2]

In [14]:
# Get unique variables from list variables
variables_uniq = set(vars_new)
variables_uniq

{'Air Moisture Content A',
 'Air temperature A',
 'Downward Solar Radiation',
 'Downward Terrestrial Rad',
 'Incremental Precipitation A',
 'Incremental Precipitation B',
 'Incremental Precipitation C',
 'Scalar Wind Speed A',
 'Scalar Wind Speed C',
 'Snow Depth A',
 'Snow Depth QC value',
 'Soil Heat Flux  A',
 'Soil Moisture A',
 'Soil Temperature A',
 'Soil Temperature B',
 'Soil Temperature C',
 'Total Pressure Adjusted to Sea-level',
 'Total Pressure Unadjusted A',
 'Upward Solar Radiation',
 'Upward Terrestrial Rad',
 'Wind Direction at A'}

In [15]:
# Extract data for each variable from the dictionary and create a xray.Dataset

ds_list = [] # Initalize list of xray Datasets (each a different variable)

# For each unique variable in the dictionary
for c_var in variables_uniq:
    print(c_var)
    all_vars={} # Initialize dictionary that only contains one variable for all stations
    # For each station
    for c_sta in stations_all:
        # Test if this varible was measured at this station
        if ((c_sta,c_var) in c_dict):
            all_vars[c_sta] = c_dict[(c_sta,c_var)]
        else: # Variable doesn't exists at this station so pad it with -9999 (needed to merge into one netcdf file)
            index_csta = time_index[c_sta]
            df_missing = pd.DataFrame(index=index_csta, columns=[c_var])
            #df_missing = df_missing.fillna(-9999)
            all_vars[c_sta] = df_missing

    # Concatenate each variable by stations
    c_obs_all = pd.concat(all_vars,axis=0,keys=stations_all)
    #c_obs_all = pd.DataFrame(c_obs_all) # not needed
    
    # Convert to xray and add to list
    ds = xr.Dataset.from_dataframe(c_obs_all)
    # Add to list and rename variables
    ds_list.append(ds.rename({'level_0':'station','TIMESTAMP':'time'}))  

Upward Solar Radiation
Total Pressure Unadjusted A
Scalar Wind Speed C
Scalar Wind Speed A
Snow Depth A
Soil Heat Flux  A
Upward Terrestrial Rad
Incremental Precipitation B
Soil Temperature C
Soil Temperature B
Snow Depth QC value
Soil Moisture A
Downward Terrestrial Rad
Air Moisture Content A
Incremental Precipitation A
Wind Direction at A
Incremental Precipitation C
Soil Temperature A
Downward Solar Radiation
Total Pressure Adjusted to Sea-level
Air temperature A


In [16]:
# Combine all variable Datasets using xray.update()
ds_all = xr.Dataset()
[ds_all.update(c_ds) for c_ds in ds_list]
ds_all

<xarray.Dataset>
Dimensions:                               (station: 8, time: 412981)
Coordinates:
  * station                               (station) object 'CNT' 'FSR' 'HMW' ...
  * time                                  (time) datetime64[ns] 2005-03-10T15:30:00 ...
Data variables:
    Upward Solar Radiation                (station, time) float64 nan nan ...
    Total Pressure Unadjusted A           (station, time) float64 nan nan ...
    Scalar Wind Speed C                   (station, time) float64 nan nan ...
    Scalar Wind Speed A                   (station, time) float64 nan nan ...
    Snow Depth A                          (station, time) float64 nan nan ...
    Soil Heat Flux  A                     (station, time) float64 nan nan ...
    Upward Terrestrial Rad                (station, time) float64 nan nan ...
    Incremental Precipitation B           (station, time) float64 nan nan ...
    Soil Temperature C                    (station, time) float64 nan nan ...
    Soil Tempe

In [17]:
# Add variable attributes (units), and fix variable names (remove spaces)
for cvar in ds_all.data_vars:
    # add units as attributes
    ds_all.get(cvar).attrs['unit']   = units_all[cvar]
    # Remove spaces in variable names
    ds_all.rename({cvar:cvar.replace(" ","")},inplace=True)

In [18]:
metadata

,lat,long,elevation(m)
station,,,
BNS,50.820920,-115.213910,2099
PWL,50.825980,-115.198200,2100
FRS,50.838230,-115.215780,2330
CRN,50.821600,-115.206500,2205
BRP,50.760610,-115.367150,2260
FLG,50.830040,-115.228510,2565
PYT,51.685600,-116.544900,2237
BWH,51.635180,-116.490280,2421
CRG,50.821500,-115.206300,2211


In [19]:
# Get stations we have from metadata
A = metadata.index.tolist()
B = ds_all.station.values.tolist()
sta_missing = list(set(A) - set(B))
metadata.drop(sta_missing,inplace=True)
# Reindex so order of stations is same as ds_all
metadata = metadata.reindex(index=B)

In [20]:
# need to make sure order of metadata is the same as ds_all.station
print(ds_all.station.values.tolist())
print(metadata.index.tolist())

['CNT', 'FSR', 'HMW', 'LLF', 'UCT', 'UPC', 'UPF', 'VVW']
['CNT', 'FSR', 'HMW', 'LLF', 'UCT', 'UPC', 'UPF', 'VVW']


In [21]:
# Add meta data for each station
ds_all = ds_all.merge({'Elevation': ('station',metadata[' elevation(m)'])})
ds_all = ds_all.merge({'Lat': ('station',metadata[' lat'])})
ds_all = ds_all.merge({'Lon': ('station',metadata[' long'])})

In [22]:
# Make meta data coordiates from variables
ds_all.set_coords(['Elevation','Lat','Lon'], inplace=True)

<xarray.Dataset>
Dimensions:                           (station: 8, time: 412981)
Coordinates:
  * station                           (station) object 'CNT' 'FSR' 'HMW' ...
  * time                              (time) datetime64[ns] 2005-03-10T15:30:00 ...
    Elevation                         (station) float64 2.47e+03 2.325e+03 ...
    Lat                               (station) float64 50.96 50.96 50.94 ...
    Lon                               (station) float64 -115.2 -115.2 -115.1 ...
Data variables:
    UpwardSolarRadiation              (station, time) float64 nan nan nan ...
    TotalPressureUnadjustedA          (station, time) float64 nan nan nan ...
    ScalarWindSpeedC                  (station, time) float64 nan nan nan ...
    ScalarWindSpeedA                  (station, time) float64 nan nan nan ...
    SnowDepthA                        (station, time) float64 nan nan nan ...
    SoilHeatFluxA                     (station, time) float64 nan nan nan ...
    UpwardTerrestrialR

In [23]:
ds_all.time

<xarray.DataArray 'time' (time: 412981)>
array(['2005-03-10T15:30:00.000000000', '2005-03-10T15:45:00.000000000',
       '2005-03-10T16:00:00.000000000', ..., '2016-12-19T12:00:00.000000000',
       '2016-12-19T12:15:00.000000000', '2016-12-19T12:30:00.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2005-03-10T15:30:00 2005-03-10T15:45:00 ...

In [24]:
## Convert RH to specific humidity (kg/kg)
# Fill missing air pressure with mean of all stations (not great but ok for now)
PRESS = ds_all['TotalPressureUnadjustedA']*100
PRESS = PRESS.fillna(PRESS.mean())
ds_all['SPH'] = RH_2_SPHM(ds_all['AirMoistureContentA'], PRESS , ds_all['AirtemperatureA']+273.15)  
ds_all.get('SPH').attrs['unit'] = 'kg/kg'

In [25]:
# Export to netcdf
ds_all.to_netcdf(cfileout,format='netcdf4') 

In [26]:
# # Trim to nov 2014 storm
# ds_storm = ds_all.sel(time=pd.date_range(start=datetime(2014,11,25),end=datetime(2014,12,3),freq='15min'))
# ds_storm.to_netcdf(cfileout_storm,format='netcdf4') 

In [27]:
# plt.plot(ds_all.ScalarWindSpeedA.T*1000)

In [28]:
# # Bad Marmot variables rename (redo)
# var_names = {'Air Temperature_AVG':'Air temperature A',
# 'Air Temperature_Avg':'AirtemperatureA',
# 'Air Temperature Avg':'AirtemperatureA',
# 'Relative Humidity corrected_AVG':'Air Moisture Content A',
# 'Relative Humidity corrected_Avg':'Air Moisture Content A',
# 'RMY Wind Speed_AVG':'Scalar Wind Speed A',
# 'Wind Speed Avg':'Scalar Wind Speed A',
# 'MetOne Wind Speed_AVG':'Scalar Wind Speed A',
# 'RMY Wind Direction_AVG':'Wind Direction at A',
# 'Wind Direction Avg':'Wind Direction at A',
# 'MetOne Wind Direction_AVG':'Wind Direction at A',
# 'Snow Depth_AVG':'Snow Depth A',
# 'Snow Depth':'Snow Depth A',
# 'SR50 Quality_AVG':'Snow Depth QC value',
# 'Snow Depth Signal Quality':'Snow Depth QC value',
# 'Incoming Solar Radiation':'Downward Solar Radiation',
# 'Incoming Solar Radiation_AVG':'Downward Solar Radiation',
# 'Outgoing Solar Radiation_AVG':'Upward Solar Radiation',
# 'Outgoing Solar Radiation':'Upward Solar Radiation',
# 'Corrected Incoming Longwave Radiation_AVG':'Downward Terrestrial Rad',
# 'Corrected Outgoing Longwave Radiation_AVG':'Upward Terrestrial Rad',
# 'Corrected Net Radiation_AVG':'Net Radiation',
# 'IRTC Infrared Thermocouple Surface Temperature':'Surface Temperature A',
# 'Rainfall_instantaneous':'Incremental Precipitation B',
# 'Rainfall_Incremental':'Incremental Precipitation B',
# 'Albedo_AVG':'Albedo',
# 'VWC':'Soil Moisture A',
# 'Soil Temperaure_5cm below surface_AVG':'Soil Temperature A',
# 'Soil Temperaure_5cm below surface Avg':'Soil Temperature A',
# 'Soil Heat Flux_AVG':'Soil Heat Flux  A',
# 'Soil Temperaure_10cm below surface_AVG':'Soil Temperature B',
# 'Soil Temperaure_15cm below surface Avg':'Soil Temperature B',
# 'Soil Temperaure_15cm below surface_AVG':'Soil Temperature B',
# 'Soil Temperaure_20cm below surface_AVG':'Soil Temperature C',
# 'Soil Temperaure_25cm below surface_AVG':'Soil Temperature C',
# 'Soil Temperaure_40cm below surface_AVG':'Soil Temperature D',
# '3Cup Wind Speed_AVG':'Scalar Wind Speed B',
# 'Accumulated All Type Precipitation_mm_AVG':'Incremental Precipitation A',
# 'Barometric Pressure':'Total Pressure Unadjusted A',
# 'RMY Wind Direction standard deviation':'drop1',
# 'cnr4_Sensor Temperature_AVG':'drop2',
# 'Wind Direction Standard Deviation':'drop3',
# 'Wind Speed_Max':'drop4',
# 'MetOne Wind Direction standard deviation':'drop5',
# 'MetOne Sonic Temperature_AVG':'drop6'}